In [30]:
from database import engine
from model import Base
Base.metadata.create_all(bind=engine)

In [31]:
# Base.metadata.drop_all(engine)

In [32]:
from pinecone_text.sparse import BM25Encoder

In [33]:
import os
from dotenv import load_dotenv
from langchain.agents import AgentExecutor
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
from tools.retriver_tool import create_retriever_tool
from document_processor import process_new_documents

In [34]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_core.prompts import MessagesPlaceholder

In [35]:
load_dotenv()    
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API")

In [36]:
document_retriever = create_retriever_tool(
    index_name="db",
    tool_name="document_retriever",
    description="Search knowledge base for document content relevant to a query"
)

In [37]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a professional document analyst assistant. Use tools to find relevant information."),
    ("human", "{input}"),
    MessagesPlaceholder("agent_scratchpad")
])

llm = ChatGroq(model="openai/gpt-oss-120b")

# retrieval_chain = create_stuff_documents_chain(llm,prompt)
agent = create_tool_calling_agent(llm, [document_retriever], prompt)

agent_executor = AgentExecutor(
    agent=agent,
    tools=[document_retriever],
    verbose=True,
    handle_parsing_errors=True
)

In [38]:
from file_scanner import scan_directory
def process_and_query(query: str, directory: str = None) -> str:
    if directory:
        scan_directory(directory)
        print("Documents queued for background processing...")
    
    result = agent_executor.invoke({"input": query})
    return result['output']

In [40]:
response = process_and_query("Authors of attention is all you need?", "/home/thkr1408/Projects/Inquiro/uploads/")
print(response)

File path:  /home/thkr1408/Projects/Inquiro/uploads/attention.pdf
Skipping processed file: attention.pdf
File path:  /home/thkr1408/Projects/Inquiro/uploads/speech.txt
Skipping processed file: speech.txt
File path:  /home/thkr1408/Projects/Inquiro/uploads/pdfs
File path:  /home/thkr1408/Projects/Inquiro/uploads/images
File path:  /home/thkr1408/Projects/Inquiro/uploads/docs
Documents queued for background processing...


> Entering new AgentExecutor chain...

Invoking: `document_retriever` with `Attention Is All You Need authors`


Full attentions for head 5. Bottom: Isolated attentions from just the word ‘its’ for attention heads 5
and 6. Note that the attentions are very sharp for this word.
14
[Source: /home/thkr1408/Projects/Inquiro/uploads/attention.pdf, Page: 13]

the approach we take in our model.
As side benefit, self-attention could yield more interpretable models. We inspect attention distributions
from our models and present and discuss examples in the appendix. Not only do 